<a href="https://colab.research.google.com/github/joaovictormtl/Data-Science/blob/main/projeto_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação

In [ ]:
import pandas as pd
import gdown

url = 'https://drive.google.com/uc?id=1M5Xb8XQ0efRCqJW8WJ64Rhdo6jOjHBQU'
output = 'age_dataset.csv'
gdown.download(url, output, quiet=False)
df = pd.read_csv(output)

In [ ]:
df.head(1)

#Polimento

In [ ]:
df = df.rename(columns={'Id':'id', 'Name':'nome', 'Short description':'descricao',\
                   'Gender':'genero','Country':'pais', 'Occupation':'profissao',\
                   'Birth year':'nascimento', 'Death year':'ano_morte',\
                   'Manner of death':'morte_causa', 'Age of death':'idade_morte'})

##Mudando o número máximo de caracteres para as colunas

In [ ]:
df['descricao'].str.len().max()

269.0

In [ ]:
pd.set_option('max_colwidth', 250)

In [ ]:
df.head(1)

,id,nome,descricao,genero,pais,profissao,nascimento,ano_morte,morte_causa,idade_morte
0,Q23,George Washington,1st president of the United States (1732–1799),Male,United States of America; Kingdom of Great Britain,Politician,1732,1799.0,natural causes,67.0


# Valores Nulos

In [ ]:
df.isnull().sum()

id                   0
nome                 0
descricao        67900
genero          133646
pais            335509
profissao       206914
nascimento           0
ano_morte            1
morte_causa    1169406
idade_morte          1
dtype: int64

## Mudando o tipo primitivo das colunas e Preenchendo Valores Nulos

In [ ]:
df['descricao'].fillna("Não Preenchido", inplace=True)

In [ ]:
df['genero'].fillna("Não Preenchido", inplace=True)

In [ ]:
df['pais'].fillna("Não Preenchido", inplace=True)

In [ ]:
df['profissao'].fillna("Não Preenchido", inplace=True)

In [ ]:
df['ano_morte'].fillna(0, inplace=True)
df['ano_morte'] = df.ano_morte.astype('int')

In [ ]:
df['idade_morte'].fillna(0, inplace=True)
df['idade_morte'] = df.idade_morte.astype('int')

In [ ]:
df['morte_causa'].fillna("Desconhecido", inplace=True)

In [ ]:
df.isna().sum()

id             0
nome           0
descricao      0
genero         0
pais           0
profissao      0
nascimento     0
ano_morte      0
morte_causa    0
idade_morte    0
dtype: int64

# Manipulações

In [ ]:
df2 = df

## Períodos Históricos

In [ ]:
def definirPeriodo(row):
    if row['ano_morte'] < 476:
        return "Antigo"
    elif row['nascimento'] >= 476 and row['ano_morte'] < 1453:
        return "Médio"
    elif row['nascimento'] >= 1453 and row['ano_morte'] < 1789:
        return "Moderno"
    else:
        return "Contemporâneo"

In [ ]:
df['periodo'] = df.apply(definirPeriodo, axis=1)

In [ ]:
df.head(5)

## 1 - Média da idade com que as pessoas morriam nos diferentes períodos históricos

In [ ]:
periodo_morte = df.groupby('periodo').mean()['idade_morte']
periodo_morte.to_frame().sort_values(by='idade_morte', ascending=False)

## 2 - Suicídio: Média da idade com que as pessoas se suicidam

In [ ]:
selecao = df['morte_causa'] == 'suicide'
df_suicide = df[selecao]

df_suicide.groupby('morte_causa').mean()['idade_morte'].to_frame()

## 3 - Quantidade de Pessoas com Gêneros Diferentes


In [ ]:
df_generos_diferentes = df.groupby('genero').count().reset_index().sort_values('id', ascending=False)[['genero', 'id']]
df_generos_diferentes = df_generos_diferentes.loc[df_generos_diferentes['genero'] != 'Male', :]
df_generos_diferentes = df_generos_diferentes.loc[df_generos_diferentes['genero'] != 'Female', :]
df_generos_diferentes = df_generos_diferentes.loc[df_generos_diferentes['genero'] != 'Não Preenchido', :]

## 4 - Homicídio: Profissionais mais sujeitos a sofrer homicídios

In [ ]:
df_homicide = df.query('morte_causa == "homicide"')

In [ ]:
df_homicide = df_homicide.groupby('profissao').count().reset_index().sort_values('id', ascending=False)[['profissao', 'id']]

In [ ]:
df_homicide['id'] = df_homicide.id.astype('int')

In [ ]:
df_homicide = df_homicide.loc[df_homicide['id'] > 20, :]
df_homicide.head()

# Dash

In [ ]:
pip install dash

In [ ]:
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import plotly.graph_objects as go

# Média da Idade com que as pessoas morriam nos diferentes períodos históricos

In [ ]:
lista_media_mortes = list(df.groupby('periodo').mean()['idade_morte'])
per = list(df['periodo'].unique())
periodos = sorted(per)

# Criando o DataFrame com as informações necessárias

df_periodos = pd.DataFrame({
    "Período Histórico": periodos,
    "Média Idade": lista_media_mortes
})

# Profissões que mais possuem suicidas

In [ ]:
# Agrupar por profissão e contar o número de suicídios em cada profissão
suicide_por_profissao = df_suicide.groupby('profissao').size().reset_index(name='quantidade')

# Ordenar por quantidade de suicídios em ordem decrescente
suicide_por_profissao = suicide_por_profissao.sort_values(by='quantidade', ascending=False)
suicide_por_profissao_filtrado = suicide_por_profissao[suicide_por_profissao['quantidade'] > 20]

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=external_stylesheets)

fig = px.bar(df_periodos, x="Período Histórico", y="Média Idade", color="Período Histórico")
# Opções do Dropdown
opcoes = list(df_periodos['Período Histórico'])
opcoes.append("Todos")

fig2 = px.bar(suicide_por_profissao_filtrado, x='profissao', y='quantidade', title="Profissao x Quantidade de Suicidios")
# Suicidas: Profissões
profissoes = list(suicide_por_profissao_filtrado['profissao'])
profissoes.append("Todas")

fig3 = go.Figure(data=[go.Pie(labels=df_generos_diferentes['genero'], values=df_generos_diferentes['id'])])

fig4 = px.bar(df_homicide, x="profissao", y='id', color="profissao")

app.layout = html.Div(children = [
    html.H1(children = "Média das Idades", style={"margin-top":'50px'}),
    html.Div(children = '''
      Este gráfico mostra a média da idade com que as pessoas morriam nos diferentes períodos históricos.
    '''),

    html.Div(style={"margin-top":'20px'}),
    dcc.Dropdown(opcoes, value='Todos', id="lista_periodos"),

    dcc.Graph(
        id="grafico_periodos",
        figure=fig
    ),

    html.H1(children="Profissões: Quatidade de Suicidas", style={"margin-top":'50px'}),
    html.Div(children='''
    Este gráfico mostra a quantidade de suicidas entre as diferentes profissões.'''
    ),

    html.Div(style={"margin-top":'20px'}),
    dcc.Dropdown(profissoes, value='Todas', id='lista_profissoes'),

    dcc.Graph(
        id="grafico_prof_suicide",
        figure=fig2
    ),

    html.H1(children="Quantidade de Pessoas com Gêneros Diferentes", style={"margin-top":'50px'}),
    html.Div(children="Este gráfico mostra a quantidade de pessoas que possuem o gênero diferente de: Masculino e Feminino."),

    dcc.Graph(
        id="grafico_generos",
        figure=fig3
    ),

    html.H1(children="Homicídio", style={"margin-top":'50px'}),
    html.Div(children="Este gráfico mostra os profissionais mais propensos a sofrer homicídio."),

    dcc.Graph(
        id="grafico_homicide",
        figure=fig4
    )

])

# Períodos
@app.callback(
    Output('grafico_periodos', 'figure'),
    Input('lista_periodos', 'value')
)
def update_output(value):
  if value == "Todos":
    fig = px.bar(df_periodos, x="Período Histórico", y="Média Idade", color="Período Histórico")
  else:
    df_filtragem = df_periodos.loc[df_periodos['Período Histórico'] == value, :]
    fig = px.bar(df_filtragem, x="Período Histórico", y="Média Idade", color="Período Histórico")
  return fig

# Suicidas: Profissões
@app.callback(
    Output('grafico_prof_suicide', 'figure'),
    Input('lista_profissoes', 'value')
)
def update_profissao(value):
  if value == 'Todas':
    fig2 = px.bar(suicide_por_profissao_filtrado, x='profissao', y='quantidade', title="Profissao x Quantidade de Suicidios")
  else:
    df_filtragem = suicide_por_profissao_filtrado.loc[suicide_por_profissao_filtrado['profissao'] == value, :]
    fig2 = px.bar(df_filtragem, x='profissao', y='quantidade', title="Profissao x Quantidade de Suicidios")
  return fig2

if __name__ == '__main__':
  app.run_server(debug=True)

<IPython.core.display.Javascript object>